In [1]:
# --- 0. Libraries ---
from pydantic import BaseModel
from typing import Literal, Any, Dict
import json
import uuid

In [2]:
# --- 1. A2A and MCP Data Models ---

print("✅ Defining A2A/MCP Data Models...")
print("-----------------------------------")

# RAW SENSOR DATA (Internal formats unique to each Agent)
class RadarData(BaseModel):
    # Agent 1 format: Polar coordinates
    range_meters: float
    azimuth_degrees: float

class VisualData(BaseModel):
    # Agent 2 format: Classification and certainty
    classification: Literal["drone", "aircraft", "vessel"]
    certainty_percent: int

# NORMALIZED DATA (The standardized output of the MCP Tool)
class NormalizedTarget(BaseModel):
    # Standardized format: Cartesian coordinates and normalized confidence
    x_pos: float # Normalized coordinate
    y_pos: float # Normalized coordinate
    target_type: str
    confidence: float # Normalized 0.0 to 1.0

# A2A PROTOCOL DATA
class A2ATask(BaseModel):
    """Simulates an A2A Task sent from FusionAgent (Client) to SensorAgents (Servers)."""
    task_id: str = str(uuid.uuid4())
    sector_id: str
    request_type: str = "track"

class A2AArtifact(BaseModel):
    """Simulates the final structured A2A Artifact (Report) generated by the FusionAgent."""
    fusion_id: str
    targets: list[NormalizedTarget]
    summary: str

✅ Defining A2A/MCP Data Models...
-----------------------------------


In [3]:
# --- 2. MCP Tool Implementation ---

def data_normalization_tool(radar_data: RadarData, visual_data: VisualData) -> NormalizedTarget:
    """
    Simulates the Model Context Protocol (MCP) Tool.
    Takes disparate raw data and outputs a single, standardized NormalizedTarget.
    """
    print("   [MCP Tool] Executing data normalization logic...")
    
    # 1. Normalize position (Example: Convert polar to a simplified Cartesian grid)
    range_m = radar_data.range_meters
    azimuth_rad = radar_data.azimuth_degrees * 3.14159 / 180.0
    
    # R * cos(theta) for X, R * sin(theta) for Y
    x = range_m * 0.5 # Simplified/Placeholder calculation for X
    y = range_m * 0.7 # Simplified/Placeholder calculation for Y

    # 2. Normalize classification/certainty
    confidence = visual_data.certainty_percent / 100.0 # Convert 0-100% to 0.0-1.0
    target_type = visual_data.classification.upper()
    
    print(f"   [MCP Tool] Normalized position: ({round(x, 2)}, {round(y, 2)})")
    
    # Return the standardized model
    return NormalizedTarget(
        x_pos=x,
        y_pos=y,
        target_type=target_type,
        confidence=confidence
    )

In [4]:
# --- 3. Simulated A2A Agents ---

class SensorAgent:
    """Simulates a Remote Agent (Server) that processes an A2A Task."""
    def __init__(self, agent_id: str):
        self.agent_id = agent_id

    # This method simulates the A2A HTTP endpoint that receives a TASK
    def process_task(self, task: A2ATask) -> Any:
        print(f"\n[{self.agent_id} | A2A Server] Received Task ID {task.task_id[:8]} for Sector {task.sector_id}")
        
        # Simulated Data Retrieval based on agent type
        if self.agent_id == "Radar Agent":
            # Returns RadarData (Agent 1's unique format)
            return RadarData(range_meters=1500.0, azimuth_degrees=45.0)
        elif self.agent_id == "Visual Agent":
            # Returns VisualData (Agent 2's unique format)
            return VisualData(classification="drone", certainty_percent=95)
        
        return None

class FusionAgent:
    """
    Simulates the Client Agent (Orchestrator) that uses A2A and MCP.
    """
    def __init__(self, radar_agent: SensorAgent, visual_agent: SensorAgent):
        self.radar_agent = radar_agent
        self.visual_agent = visual_agent

    def execute_fusion(self, sector: str) -> A2AArtifact:
        print("\n--- Fusion Agent Orchestration Starting ---")
        
        # --- A2A STEP 1: Request data from Sensor Agents ---
        task = A2ATask(sector_id=sector)
        
        print("\n[Fusion Agent] Sending A2A Tasks to Sensors...")
        radar_raw = self.radar_agent.process_task(task)
        visual_raw = self.visual_agent.process_task(task)
        
        print("\n[Fusion Agent] Received raw data from both sensors.")
        
        # --- MCP STEP: Normalize the combined data ---
        print("\n[Fusion Agent] Transitioning to MCP Tool Call...")
        normalized_target = data_normalization_tool(radar_raw, visual_raw)
        
        # --- A2A STEP 2: Generate the final Artifact ---
        fusion_id = f"RPT-{str(uuid.uuid4())[:8]}"
        report = A2AArtifact(
            fusion_id=fusion_id,
            targets=[normalized_target],
            summary=f"Track successful for Sector {sector}. Found 1 target of type {normalized_target.target_type} at relative normalized position."
        )
        
        print("\n[Fusion Agent] Successfully generated Final A2A Artifact.")
        return report

In [5]:
# --- 4. Execution ---

# Initialize the Agents
radar = SensorAgent("Radar Agent")
visual = SensorAgent("Visual Agent")
fusion = FusionAgent(radar, visual)

# Execute the full cycle
final_report = fusion.execute_fusion("Alpha Sector")

print("\n\n#################################################")
print("🚀 FINAL A2A ARTIFACT (Fusion Report)")
print("#################################################")
# Use Pydantic's built-in JSON export for clean output
print(final_report.model_dump_json(indent=2))

# Optional: Display the Normalized Target coordinates
print("\n--- Target Data Summary ---")
target = final_report.targets[0]
print(f"Target Type: {target.target_type}")
print(f"Confidence: {target.confidence * 100:.2f}%")
print(f"Normalized X/Y: ({round(target.x_pos, 2)}m, {round(target.y_pos, 2)}m)")


--- Fusion Agent Orchestration Starting ---

[Fusion Agent] Sending A2A Tasks to Sensors...

[Radar Agent | A2A Server] Received Task ID fc03583a for Sector Alpha Sector

[Visual Agent | A2A Server] Received Task ID fc03583a for Sector Alpha Sector

[Fusion Agent] Received raw data from both sensors.

[Fusion Agent] Transitioning to MCP Tool Call...
   [MCP Tool] Executing data normalization logic...
   [MCP Tool] Normalized position: (750.0, 1050.0)

[Fusion Agent] Successfully generated Final A2A Artifact.


#################################################
🚀 FINAL A2A ARTIFACT (Fusion Report)
#################################################
{
  "fusion_id": "RPT-8a65979a",
  "targets": [
    {
      "x_pos": 750.0,
      "y_pos": 1050.0,
      "target_type": "DRONE",
      "confidence": 0.95
    }
  ],
  "summary": "Track successful for Sector Alpha Sector. Found 1 target of type DRONE at relative normalized position."
}

--- Target Data Summary ---
Target Type: DRONE
Confidence: 